In [ ]:
import cv2
import face_recognition as fr
import os
import numpy as np

# Path to the main folder containing subfolders, each representing a different person
main_folder = "images"

# Load known faces and their names dynamically from the subfolders
known_face_encodings = []
known_face_names = []

# Traverse through all subfolders in the main folder
for person_folder in os.listdir(main_folder):
    person_folder_path = os.path.join(main_folder, person_folder)
    if os.path.isdir(person_folder_path):
        # List all image files within the subfolder
        image_files = [f for f in os.listdir(person_folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        # Load images and extract face encodings for each image
        for image_file in image_files:
            image_path = os.path.join(person_folder_path, image_file)
            image = fr.load_image_file(image_path)
            # Attempt to find faces in the image
            try:
                face_locations = fr.face_locations(image)
                if len(face_locations) > 0:
                    # Extract face encoding if a face is found
                    face_encoding = fr.face_encodings(image, face_locations)[0]
                    known_face_encodings.append(face_encoding)
                    # Associate each face encoding with the name of the image file (without extension)
                    known_face_names.append(os.path.splitext(image_file)[0])
                else:
                    print(f"No faces found in {image_file}")
            except KeyboardInterrupt:
                # If KeyboardInterrupt occurs, break out of the loop
                break

# Start video capture
video_capture = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to capture frame from camera.")
            break

        rgb_frame = frame[:, :, ::-1]
        fc_locations = fr.face_locations(rgb_frame)
        fc_encodings = fr.face_encodings(rgb_frame, fc_locations)

        for (top, right, bottom, left), face_encoding in zip(fc_locations, fc_encodings):
            matches = fr.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # If a match is found, use the name of the corresponding image file (without extension)
            if True in matches:
                match_index = matches.index(True)
                name = known_face_names[match_index]

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        cv2.imshow('Face Detection System', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    pass

video_capture.release()
cv2.destroyAllWindows()
